In [100]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys
import matplotlib
matplotlib.use('Agg')

import pandas
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

from tqdm import tqdm

In [80]:
path = "./USvideos.csv"
df = pd.read_csv(path ,  sep = ',')
df.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [82]:
print(df.tags)

0                                          SHANtell martin
1        last week tonight trump presidency|"last week ...
2        racist superman|"rudy"|"mancuso"|"king"|"bach"...
3        rhett and link|"gmm"|"good mythical morning"|"...
4        ryan|"higa"|"higatv"|"nigahiga"|"i dare you"|"...
                               ...                        
40944    aarons animals|"aarons"|"animals"|"cat"|"cats"...
40945                                               [none]
40946    I gave safiya nygaard a perfect hair makeover ...
40947    Black Panther|"HISHE"|"Marvel"|"Infinity War"|...
40948        call of duty|"cod"|"activision"|"Black Ops 4"
Name: tags, Length: 40949, dtype: object


In [127]:
df.head(1)
# df.tags.info


,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,features
0,22,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,0


In [84]:
df = df.drop(columns=['video_id', 'trending_date', 'title', 'publish_time', 'publish_time','channel_title'])
df.corr()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13588\3758889337.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,category_id,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed
category_id,1.000000,-0.168231,-0.173921,-0.033547,-0.076307,0.048949,-0.013506,-0.030011
views,-0.168231,1.000000,0.849177,0.472213,0.617621,0.002677,0.015355,-0.002256
likes,-0.173921,0.849177,1.000000,0.447186,0.803057,-0.028918,-0.020888,-0.002641
dislikes,-0.033547,0.472213,0.447186,1.000000,0.700184,-0.004431,-0.008230,-0.001853
comment_count,-0.076307,0.617621,0.803057,0.700184,1.000000,-0.028277,-0.013819,-0.003725
comments_disabled,0.048949,0.002677,-0.028918,-0.004431,-0.028277,1.000000,0.319230,-0.002970
ratings_disabled,-0.013506,0.015355,-0.020888,-0.008230,-0.013819,0.319230,1.000000,-0.001526
video_error_or_removed,-0.030011,-0.002256,-0.002641,-0.001853,-0.003725,-0.002970,-0.001526,1.000000


In [85]:
from sklearn.preprocessing import MinMaxScaler
datatypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
normarization = df.select_dtypes(include=datatypes)
for col in normarization.columns:
    MinMaxScaler(col)

In [86]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10)
features = kmeans.fit_predict(normarization)
df['features'] = features
MinMaxScaler(df['features'])

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


MinMaxScaler(feature_range=0        0
1        5
2        5
3        0
4        5
        ..
40944    0
40945    0
40946    0
40947    9
40948    9
Name: features, Length: 40949, dtype: int32)

In [91]:
path = "./USvideos.csv"
df2 = pd.read_csv(path ,  sep = ',')
df2.columns


Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [125]:
search_string = 'WE WANT TO TALK ABOUT OUR MARRIAGE'
filtered_df = df2[df2['title'].str.contains(search_string, case=False, na=False)]


# filtered_df = df2[df2['title'] == search_string]
# filtered_df

In [126]:
class Spotify_Recommendation():
    def __init__(self, dataset):
        self.dataset = dataset

    def recommend(self, songs, amount=1):
        distance = []
        rec = self.dataset[self.dataset.title.str.lower() != songs.lower()]  # Filter out the input song
        
        # Check if there are songs left after filtering
        if rec.empty:
            print("No matching songs found in the dataset.")
            return None  # or handle the empty case accordingly
        
        song = rec.head(1).values[0]
# your videos columns id bystring
        for song_row in tqdm(rec.values):
            d = 0
            for col_idx, col_value in enumerate(song):
                if col_idx not in [1, 6, 12, 14, 18] and pd.api.types.is_numeric_dtype(rec.iloc[:, col_idx]):
                    d = d + np.absolute(float(col_value) - float(song_row[col_idx]))
# if else your videos columns id by numbers
            # for col in np.arange(len(rec.columns)):
            #     if col not in [1, 6, 12, 14, 18]:
            #         d = d + np.absolute(float(song[col]) - float(song_row[col]))
            distance.append(d)
        
        rec['distance'] = distance
        rec = rec.sort_values('distance')
        columns = ['channel_title', 'title','thumbnail_link']
        return rec[columns][:amount]

# Assuming df2 is your DataFrame
recommendations = Spotify_Recommendation(df2)
recommendations.recommend("WE WANT TO TALK ABOUT OUR MARRIAGE", 10)


100%|██████████| 40942/40942 [00:22<00:00, 1820.22it/s]
C:\Users\Admin\AppData\Local\Temp\ipykernel_13588\1207876697.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec['distance'] = distance


,channel_title,title,thumbnail_link
1,LastWeekTonight,The Trump Presidency: Last Week Tonight with J...,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg
29045,jeffreestar,*NEW* FENTY BEAUTY BODY LAVA & FAIRY BOMB | HI...,https://i.ytimg.com/vi/XWrB4qOF_UA/default.jpg
14509,PlayStation,God of War – Story Trailer | PS4,https://i.ytimg.com/vi/K0u_kAWLJOA/default.jpg
26990,Marvel Entertainment,Marvel Studios' Avengers: Infinity War - All o...,https://i.ytimg.com/vi/YZ1UN5gvLjQ/default.jpg
29340,AlishaMarie,Funny Pranks for April Fools!! Roommate Wars,https://i.ytimg.com/vi/RiGFGXd9Xvs/default.jpg
26740,Casually Explained,Casually Explained: Men's Fashion,https://i.ytimg.com/vi/KyQyz2upSgY/default.jpg
32433,First We Feast,Alton Brown Rigorously Reviews Spicy Wings | H...,https://i.ytimg.com/vi/T1-k7VYwsHg/default.jpg
37847,LaurDIY,DOLLAR STORE DIY CHALLENGE,https://i.ytimg.com/vi/M79o2UejYKo/default.jpg
30797,Rosanna Pansino,Whats In My HUGE Fridge?,https://i.ytimg.com/vi/we5djgQwK9w/default.jpg
36388,FaZe Rug,I pranked him... then I BOUGHT him his DREAM C...,https://i.ytimg.com/vi/2v4-L4PkV9U/default.jpg
